In [25]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV, ShuffleSplit, KFold, train_test_split, LeaveOneOut
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from itertools import permutations
import multiprocessing

# MFCC combinations that delivered the best results upon 10-fold CV (1 is the original MFCCs and 2 and 3
# are the first and second order derivatives)
best_convnets_combinations_for_task = {
    1: [1, 2],
    2: [1, 2],
    3: [1, 2]
}

classification_tasks = ['decade', 'artist', 'album']

def convnet_from_layers(full_convnet, layers):
        custom_convnet = []
        for layer in layers:
            custom_convnet.extend(full_convnet[(40*(layer-1)):(40 *layer)])
        return custom_convnet

def grid_search_and_classification_report(target_idx):
    n_cpu = multiprocessing.cpu_count()
    print(" ##################################### ")
    print(" ########## %s classifier #########" % classification_tasks[target_idx-1])
    print(" ##################################### ")
    
    # Set the parameters for cross-validation
    tuned_parameters = [{'kernel': ['rbf'], "gamma": [0.5 ** i for i in [1, 3, 5, 7, 9, 11]] + ['auto'],
                         'C': [0.1, 2.0, 8.0, 32.0]},
                        {'kernel': ['linear'], 'C': [0.1, 2.0, 8.0, 32.0]}]  
    print()

    with open('/usr/app/data/features_task_%i.txt' % (i)) as features, \
         open('/usr/app/data/labels_task_%i.txt' % (i)) as labels_list:
        labels = labels_list.readlines()
        full_features = features.readlines()
        # Scaling features
        mfcc_features = [convnet_from_layers([float(feature) for feature in feature_set.split()], best_convnets_combinations_for_task[target_idx])  
                                            for feature_set in full_features]
        scaler = StandardScaler()
        mfcc_features = scaler.fit_transform(mfcc_features)
        print("######################### SVC ##############################")
        clf = GridSearchCV(
            svm.SVC(class_weight='balanced'), tuned_parameters, cv=10, n_jobs=n_cpu, scoring='precision_macro'
        ).fit(mfcc_features, labels)
        print("Best parameters set found on development set:")
        print()
        print(clf.best_params_)
        print()
        print('     -> best score {}'.format(clf.best_score_))
        print()        

def warn(*args, **kwargs):
    pass

import warnings
warnings.warn = warn
# Running grid search for each classificationt task
if __name__ == "__main__":
    for i in [2, 3]:
        grid_search_and_classification_report(i)

 ##################################### 
 ########## artist classifier #########
 ##################################### 

######################### SVC ##############################
Best parameters set found on development set:

{'kernel': 'rbf', 'gamma': 0.125, 'C': 2.0}

     -> best score 0.7919365961419936

 ##################################### 
 ########## album classifier #########
 ##################################### 

######################### SVC ##############################
Best parameters set found on development set:

{'kernel': 'rbf', 'gamma': 0.03125, 'C': 8.0}

     -> best score 0.8815324405964405

